In [1]:
import duckdb
from pathlib import Path
from dotenv import load_dotenv
import os

In [2]:
dotenv_path = Path("/Users/arthurgonzaga/DataKick/.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [7]:

# Configuração das credenciais do MinIO
MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY')
MINIO_ENDPOINT = "localhost:9000"

# Caminho do arquivo no MinIO
minio_csv_path = "s3://bronze/datadrop/kaggle/database.csv"

minio_bronze_path = "s3://bronze/api_futebol/campeonato_brasileiro/1a_rodada/*.json"
# minio_bronze_path = "https://bronze.s3.amazonaws.com/api_futebol/campeonato_brasileiro/1a_rodada/*.json"

In [5]:

# Conectando ao DuckDB
con = duckdb.connect()

config = f"""
INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='{MINIO_ACCESS_KEY}';
SET s3_secret_access_key='{MINIO_SECRET_KEY}';
SET s3_endpoint='{MINIO_ENDPOINT}'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO
"""

# Configurando as credenciais e o endpoint do MinIO no DuckDB
con.execute(config)

In [6]:
print(config)


INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='jknVRF5166kyQRLiK7gc';
SET s3_secret_access_key='OKczJuafJxBfAQ8xn9hiHguhbQtQgmYALDwLY1Sy';
SET s3_endpoint='localhost:9000'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO



In [ ]:
# query = f"""
# SELECT
#     Jogador AS player,
#     Time AS team,
#     "#" AS number,
#     Nação AS nationality,
#     "Pos." AS position,
#     Idade AS age,
#     "Min." AS minutes,
#     Gols AS goals,
#     "Assis." AS assists,
#     PB AS progressive_passes,
#     PT AS total_passes,
#     TC AS tackles,
#     CaG AS goals_conceded,
#     CrtsA AS yellow_cards,
#     CrtV AS red_cards,
#     Contatos AS touches,
#     Div AS duels,
#     Crts AS fouls,
#     Bloqueios AS blocks,
#     xG AS expected_goals,
#     npxG AS non_penalty_expected_goals,
#     xAG AS expected_assists,
#     SCA AS shot_creating_actions,
#     GCA AS goal_creating_actions,
#     Cmp AS passes_completed,
#     Att AS passes_attempted,
#     "Cmp%" AS pass_completion_percentage,
#     PrgP AS progressive_passes_distance,
#     Conduções AS dribbles,
#     PrgC AS progressive_carries,
#     Tent AS dribbles_attempted,
#     Suc AS dribbles_successful,
#     Data AS updated_date
# FROM read_csv_auto('{minio_csv_path}')
# """

In [8]:
query = f'''
SELECT * 
FROM read_json(
    '{minio_bronze_path}',
    format='auto'
);
'''

In [17]:
query = f'''
WITH partida AS (
   SELECT 
        -- Informações gerais da partida
        json_extract(partida, '$.partida_id')::TEXT AS partida_id,
        json_extract(partida, '$.data_realizacao')::TEXT AS data_realizacao,
        json_extract(partida, '$.hora_realizacao')::TEXT AS hora_realizacao,
        json_extract(partida, '$.placar')::TEXT AS placar,
        
        -- Informações do campeonato
        json_extract(partida, '$.campeonato.nome')::TEXT AS campeonato_nome,
        json_extract(partida, '$.campeonato.edicao.nome')::TEXT AS campeonato_edicao,
        json_extract(partida, '$.campeonato.edicao.temporada')::TEXT AS campeonato_temporada,
        json_extract(partida, '$.campeonato.rodada_atual.nome')::TEXT AS rodada_nome,
        
        -- Times mandante e visitante
        json_extract(partida, '$.time_mandante.nome_popular')::TEXT AS mandante_nome,
        json_extract(partida, '$.time_visitante.nome_popular')::TEXT AS visitante_nome,
        
        -- Placar dos times
        json_extract(partida, '$.placar_mandante')::TEXT AS placar_mandante,
        json_extract(partida, '$.placar_visitante')::TEXT AS placar_visitante,
        
        -- Estatísticas do mandante
        json_extract(partida, '$.estatisticas.mandante.posse_de_bola')::TEXT AS mandante_posse,
        json_extract(partida, '$.estatisticas.mandante.faltas')::TEXT AS mandante_faltas,
        json_extract(partida, '$.estatisticas.mandante.finalizacao.total')::TEXT AS mandante_finalizacoes,
        
        -- Estatísticas do visitante
        json_extract(partida, '$.estatisticas.visitante.posse_de_bola')::TEXT AS visitante_posse,
        json_extract(partida, '$.estatisticas.visitante.faltas')::TEXT AS visitante_faltas,
        json_extract(partida, '$.estatisticas.visitante.finalizacao.total')::TEXT AS visitante_finalizacoes,
        
        -- Substituições
        json_extract(partida, '$.substituicoes.mandante') AS subs_mandante,
        json_extract(partida, '$.substituicoes.visitante') AS subs_visitante
        
    FROM 
        read_json_auto('{minio_bronze_path}') AS partida
)

SELECT *
FROM partida;
'''

In [23]:
# Carregando o CSV do MinIO para uma tabela temporária e lendo os dados
try:
    result = con.execute(query).df()
except Exception as e:
    print("Erro ao acessar o MinIO:", e)

result

,partida_id,data_realizacao,hora_realizacao,placar,campeonato_nome,campeonato_edicao,campeonato_temporada,rodada_nome,mandante_nome,visitante_nome,placar_mandante,placar_visitante,mandante_posse,mandante_faltas,mandante_finalizacoes,visitante_posse,visitante_faltas,visitante_finalizacoes,subs_mandante,subs_visitante
0,16880,"""13/04/2024""","""18:30:00""","""Internacional 2x1 Bahia""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Internacional""","""Bahia""",2,1,"""54%""",21,12,"""46%""",13,8,"[{""saiu"":{""atleta_id"":766,""nome_popular"":""Brun...","[{""saiu"":{""atleta_id"":25700,""nome_popular"":""Es..."
1,16881,"""14/04/2024""","""17:00:00""","""Cruzeiro 3x2 Botafogo""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Cruzeiro""","""Botafogo""",3,2,"""53%""",12,10,"""47%""",10,6,"[{""saiu"":{""atleta_id"":2275,""nome_popular"":""Ram...","[{""saiu"":{""atleta_id"":3066,""nome_popular"":""Mar..."
2,16882,"""14/04/2024""","""18:30:00""","""Vitória 0x1 Palmeiras""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Vitória""","""Palmeiras""",0,1,"""52%""",12,11,"""48%""",15,13,"[{""saiu"":{""atleta_id"":11761,""nome_popular"":""Du...","[{""saiu"":{""atleta_id"":22154,""nome_popular"":""Es..."
3,16883,"""13/04/2024""","""21:00:00""","""Fluminense 2x2 Bragantino""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Fluminense""","""Bragantino""",2,2,"""60%""",8,18,"""40%""",19,13,"[{""saiu"":{""atleta_id"":2053,""nome_popular"":""Fel...","[{""saiu"":{""atleta_id"":2429,""nome_popular"":""Cle..."
4,16884,"""14/04/2024""","""16:00:00""","""Vasco 2x1 Grêmio""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Vasco""","""Grêmio""",2,1,"""49%""",17,9,"""51%""",10,5,"[{""saiu"":{""atleta_id"":2454,""nome_popular"":""Ros...","[{""saiu"":{""atleta_id"":1818,""nome_popular"":""Kan..."
5,16886,"""14/04/2024""","""16:00:00""","""Corinthians 0x0 Atlético-MG""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Corinthians""","""Atlético-MG""",0,0,"""50%""",16,9,"""50%""",15,4,"[{""saiu"":{""atleta_id"":2267,""nome_popular"":""Fag...","[{""saiu"":{""atleta_id"":2378,""nome_popular"":""Igo..."
6,16887,"""13/04/2024""","""21:00:00""","""São Paulo 1x2 Fortaleza""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""São Paulo""","""Fortaleza""",1,2,"""64%""",10,10,"""36%""",14,5,"[{""saiu"":{""atleta_id"":12100,""nome_popular"":""Ga...","[{""saiu"":{""atleta_id"":734,""nome_popular"":""Yago..."
7,16889,"""14/04/2024""","""16:00:00""","""Athletico-PR 4x0 Cuiabá""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Athletico-PR""","""Cuiabá""",4,0,"""58%""",16,17,"""42%""",9,2,"[{""saiu"":{""atleta_id"":2107,""nome_popular"":""Pab...","[{""saiu"":{""atleta_id"":19851,""nome_popular"":""De..."
8,16891,"""14/04/2024""","""16:00:00""","""Atlético-GO 1x2 Flamengo""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Atlético-GO""","""Flamengo""",1,2,"""35%""",10,10,"""65%""",10,10,"[{""saiu"":{""atleta_id"":25509,""nome_popular"":""Al...","[{""saiu"":{""atleta_id"":1817,""nome_popular"":""Eve..."
9,16892,"""13/04/2024""","""18:30:00""","""Criciúma 1x1 Juventude""","""Campeonato Brasileiro""","""Campeonato Brasileiro 2024""","""2024""","""38ª Rodada""","""Criciúma""","""Juventude""",1,1,"""46%""",12,7,"""54%""",15,10,"[{""saiu"":{""atleta_id"":7511,""nome_popular"":""Éde...","[{""saiu"":{""atleta_id"":2570,""nome_popular"":""Jea..."


In [ ]:
# Fechar a conexão
# con.close()